In [1]:
# 刪除SQL資料
"""
DELETE
  FROM [CMAPP].[dbo].[testData]
 WHERE Ddate BETWEEN '20201101' AND '20201102'
"""
# 資料寫入SQL
"""
INSERT INTO [CMAPP].[dbo].[testData]([Ddate],[KeyA],[ColA],[ColB])
VALUES ('20201104', 'A1', 5, 1.1)
"""

# 建立VIEW表
"""
CREATE VIEW [dbo].[VIEW_testData] 
AS

SELECT Ddate, KeyA, 'ColA' AS KeyB, ColA AS Value
  FROM [CMAPP].[dbo].[testData]
 UNION ALL
SELECT Ddate, KeyA, 'ColB' AS KeyB, ColB AS Value
  FROM [CMAPP].[dbo].[testData]
"""

import pandas as pd
import pyodbc

# 連線SQL
DBdata = {'ip':'192.168.99.142', 'db':'CMAPP'}
user142 = 'cmapp'
pwd142 = '0000'
conn_DBdata = pyodbc.connect('DRIVER={SQL Server};SERVER=' + DBdata['ip'] + ';DATABASE=' + DBdata['db'] + ';UID=' + user142 + ';PWD=' + pwd142)
DBdata_cr = conn_DBdata.cursor()

# 參數設定
file = r'./testData.csv'
TableName = 'testData'
dd1 = '20201101'
dd2 = '20201104'

# 刪除資料
def DelSQL(TableName, dd1, dd2):
    str_query = """
        DELETE
          FROM [CMAPP].[dbo].[{}]
         WHERE Ddate BETWEEN {} AND {}
        """
    DBdata_cr.execute(str_query.format(TableName, dd1, dd2))
    conn_DBdata.commit()
    
# 寫入資料
def InsertSQL(df, TableName):
    str_query = """
        INSERT INTO [CMAPP].[dbo].[{}]([Ddate],[KeyA],[ColA],[ColB])
        VALUES ({})"""
    insert_data = df.values.tolist()
    for i in range(len(insert_data)):
        DBdata_cr.execute(str_query.format(TableName, str(insert_data[i]).strip('[]').replace('None', 'null')))
        conn_DBdata.commit()

# 查詢資料
def QuerySQL(TableName):
    str_query = """
        SELECT *
          FROM [CMAPP].[dbo].[{}]
        """
    DBdata_cr.execute(str_query.format(TableName))
    rows = DBdata_cr.fetchall()
    conn_DBdata.commit()
    return rows

# 執行測試
df = pd.read_csv(file)  #匯入資料
df['Ddate'] = df['Ddate'].astype('str') #將資料型態轉為字串,但實際上不轉換寫入SQL時,SQL也會自動處理

DelSQL(TableName, dd1, dd2) #刪除資料

InsertSQL(df, TableName)    #寫入資料

rows = QuerySQL(TableName)  #查詢寫入資料
print(rows)


#DB斷線
conn_DBdata.close()

OperationalError: ('08001', '[08001] [Microsoft][ODBC SQL Server Driver][DBNETLIB]SQL Server does not exist or access denied. (17) (SQLDriverConnect); [08001] [Microsoft][ODBC SQL Server Driver][DBNETLIB]ConnectionOpen (Connect()). (53)')